`You will not write any code in this notebook. Write all your code in Python file genetic_algorithm.py`

In [1]:
# Install bitstring here.
import subprocess
import sys
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
install("bitstring")

In [2]:
# Set up library imports
from testing.testing import test
import bitstring
import genetic_algorithm as  GA
POPULATION_SIZE = 64
CHROMOSOME_LENGTH = 56
verbose = False

# Genetic Algorithm (Robot Wall Traversing)

In this notebook we will teach a robot how to traverse a wall. The goal of this simplified assignment is to teach you basic principles of [Genetic Algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm). Genetic Algorithms (GA) are inspired from [Evolution](https://en.wikipedia.org/wiki/Darwinism) you studied in Biology. 
You may also find these [slides](https://drive.google.com/file/d/1exXbHn_k0_jNRbRbyVrh6Ecn2SHVEvtQ/view?usp=sharing) on genetic algorithms helpful.


`The problem we will solve using GA looks as follows:`

<img src="robot_img.png" width="800">
</div>

## Scene Description:

- The simulated robot (currently at `start` facing East) has five sensors which can detect the presence or absence of wall in the five grid locations it faces, as shown. 
- It can take one of the four actions:
> Do nothing <br>
> Turn left (at 90 degrees) on the spot  <br>
> Turn right (at 90 degrees) on the spot  <br>
> Go forward one cell.  <br>
- The fitness of the robot is how many of the highlighted cells next to the wall it has covered in a fixed number of sense- act cycles.
- Robots are allowed 28 such cycles per trial, i.e., enough to follow the wall perfectly.

The Robot is allowed 28 cycles only and to perfectly traverse the wall it needs at least 28 cycles. Therefore, it should perform each step correctly. For example, even if it makes one mistake, it will waste at least 1 step; Thus, never reaching the destination in prescribed limit. 


### Encode Actions Using Bits
The robot is allowed to perform 4 actions at a time. To encode path in Python, it's not convenient to say:
```python
path = 'Step forward Step forward Step forward Turn Right Turn Right and so on'
```
Instead we can use bits to encode each of these actions. There are four actions that the robot can perfrom, and from CS-225, you should recall, to encode 4 actions we only need 2 bits. 
Note that any arbitrary encoding will work well; however, the encoding we'll use is the following:
-	00 -> Do Nothing
-	01 -> Step forward
-	10 -> Turn Right
-	11 -> Turn Left
Now, to encode the above path, the string of bits becomes:
```python
path = '0101011010...'
```
More natural for programming!


Note that to perfectly traverse the wall, the robot needs to take 28 correct steps. Therefore, To perfectly encode the correct path from start to end, fifty eight total bits are required (58=28 * 2). According to the above encoding, the correct path encoding is the following string of bits which will call Wall_Bit_String: 
```python
Wall_Bit_String = '01010101011001101101010111011001100101010101100101010101'
```
To handle this string of bits efficiently, we'll use [BitString](https://pythonhosted.org/bitstring/introduction.html) library- specifically [ConstBitStream](https://pythonhosted.org/bitstring/constbitstream.html#bitstring.ConstBitStream). All your string of bits will be wrapped in ConstBitStream.


`Aside` For the sake of this assignment, we've actually much simplified the problem (reducing solution space) by using the above encoding- this encoding will also ensure that solutions of all of you will converge (robot will reach target) in a reasonable amount of time. In practice, GAs may take a long time to converge (if they converge). Many alternative implementations are also possible. Eager students can explore them on their own. 

`An Interpertation of BitString Encoding`:
For Finding the fitness of a candidate solution, We are using a very strict condition: if a robot, makes 1 wrong move, it will not be allowed to advance any further, and its fitness so far will be returned. For example, on the contrary, a lenient condition would have been to allow the robot advance even if it makes wrong moves. 

If you decode the path the above string encodes, the robot will exactly traverse the wall in 28 steps. Our Genetic algorithm problem has to learn this exact sequence of 58 bits starting from a set of random sequence of bits called [population](https://en.wikipedia.org/wiki/Feasible_region). Note the word 'set', we'll start from `many` random sequences like this.
<br><br> 
Read the following pseudocode of GA taken from [the slides](https://drive.google.com/file/d/1exXbHn_k0_jNRbRbyVrh6Ecn2SHVEvtQ/view?usp=sharing) before moving on.
<img src="GA_alg_img.png" width="800">
</div>

Let's write that pesudocode in Python. In module you'll see the following code (equivalent of pseudocode)
```Python
def run_genetic_alg(self):
        '''  
        The pseudo you saw in slides of Genetic Algorithm is implemented here. 
        Here, You'll get a flavor of functional 
        programming in Python- Those who attempted ungraded optional tasks in tutorial
        have seen something similar there as well. 
        Those with experience in functional programming (Haskell etc)
        should have no trouble understanding the code below. Otherwise, take our word that
        this is more or less similar to the generic pseudocode in Jupyter Notebook.

        '''
        "You may not make any changes to this function."

        # Creation of Population
        solutions = self.generate_candidate_sols(self.population_size)

        # Evaluation of individuals
        parents = self.evaluate_candidates(solutions)

        while(not self.terminate):
            # Make pairs
            pairs_of_parents = self.select_parents(parents)

            # Recombination of pairs.
            recombinded_parents = list(chain(*map(lambda pair: \
                self.recombine_pairs_of_parents(pair[0], pair[1]), \
                    pairs_of_parents))) 

            # Mutation of each individual
            mutated_offspring = list(map(lambda offspring: \
                self.mutate_offspring(offspring), recombinded_parents))

            # Evaluation of individuals
            parents = self.evaluate_candidates(mutated_offspring) # new parents (offspring)
```



### `1- On Generating Candidate Solutions:`
Write your code in `generate_candidate_sols` in class `GeneticAlgorithm`. Here, you'll be creating your initial random population of size `n`. Each individual in this population is a `56 bit ConstBitStream` object. Population is a list of size `n` whose each element is a `56 bit ConstBitStream`. More information is provided in `generate_candidate_sols`.

See [Slides](https://drive.google.com/file/d/1exXbHn_k0_jNRbRbyVrh6Ecn2SHVEvtQ/view?usp=sharing) for further information. 

`Test` your implementation of `generate_candidate_sols` by running the next cell. 

Testing in this notebook is not exhaustive.

In [3]:
# Test your generate_candidate_sols() function
def generate_candidate_sols_test(generate_candidate_sols):
    test.equal(len(generate_candidate_sols(64)), (64))
    test.equal(len(generate_candidate_sols(512)), (512))
    test.equal(isinstance(generate_candidate_sols(1)[0], bitstring.ConstBitStream), True)
@test
def generate_candidate_sols(pop_size):
    genetic_algorithm = GA.GeneticAlgorithm(64, CHROMOSOME_LENGTH, verbose)
    return genetic_algorithm.generate_candidate_sols(pop_size)

### TESTING generate_candidate_sols: PASSED 3/3
###



`Program testing can be used to show the presence of bugs, but never to show their absence!`- Dijkstra.

In [4]:
isinstance(generate_candidate_sols(1)[0], bitstring.ConstBitStream)?

SyntaxError: invalid syntax (<ipython-input-4-5f9e004cadc0>, line 1)

### `2- On Evaluating Individuals:`


```Python
def evaluate_candidates(self, candidates): 
        '''
        args: candidate solutions (list) => each element is a bitstring (ConstBitStream)
        
        returns: parents (list) => each element is a bitstring (ConstBitStream) but elements 
                                   can have more than one copy. Fittest candidates will have multiple copies.
                                   Size of 'parents' must be equal to population size.  
        '''
```
In function `evaluate_candidates()`, you'll use the formula (`Roulette Wheel Selection`) to find expected number of copies of an individual in the the following generation. See [Slides.](https://drive.google.com/file/d/1exXbHn_k0_jNRbRbyVrh6Ecn2SHVEvtQ/view?usp=sharing) 

##### `Fitness of a Candidate Solution f(i, t)`:
$f(i, t)$ is fitness of individual $i$ in generation $t$. 

To find fitness of each candidate, you'll find how close it's to the target by counting matching pairs bits in 'WallBitString' and the candidate. For example, if a candidate looks like this 
```Python 
candidate     = '010101 11111001101101010111011001100101010101100101010101' # Spaces are added for clarity
WallBitString = '010101 01011001101101010111011001100101010101100101010101'
```
As you can see three pairs of this candidate match with the target (WallBitString) out of total 23 pairs. So, fitness of this candidate is $(3/28)$

You may find [read](https://pythonhosted.org/bitstring/constbitstream.html#bitstring.ConstBitStream.read) attribute of ConstBitStream class helpful.

##### `Other Terms in the Formula`:

In denominator in the formula is `average` fitness of the whole population.
We'll use this formula for finding average fitness:
```Python
total_possible_matching_pairs = 28
f_avg = (total_matching_bit_pairs_of_whole_population / total_possible_matching_pairs) / population_size
```
$n(i, t)$ is count of individual $i$ in current generation $t$. For our case, we'll ignore this for simplicity. We'll assume it's always one.

Test your implementation of `evaluate_candidates` by running the cell below.

In [5]:
# Test your evaluate_candidates() function
def evaluate_candidate_sols_test(evaluate_candidate_sols):
    genetic_algorithm = GA.GeneticAlgorithm(64, CHROMOSOME_LENGTH, verbose)
    solutions = genetic_algorithm.generate_candidate_sols(64)
    test.equal(len(evaluate_candidate_sols(genetic_algorithm, solutions)), (64))
    test.equal(isinstance(evaluate_candidate_sols(genetic_algorithm, solutions)[0], \
                          bitstring.ConstBitStream), True)
    genetic_algorithm = GA.GeneticAlgorithm(512, CHROMOSOME_LENGTH, verbose)
    solutions = genetic_algorithm.generate_candidate_sols(512)
    test.equal(len(evaluate_candidate_sols(genetic_algorithm, solutions)), (512))

@test
def evaluate_candidate_sols(genetic_algorithm, solutions):
    return genetic_algorithm.evaluate_candidates(solutions)

Fitness: [(ConstBitStream('0xbb2e7425e89e76'), 8, 0), (ConstBitStream('0x9409ae94f128f3'), 7, 0), (ConstBitStream('0xbfcf5997c1affc'), 3, 0), (ConstBitStream('0xa7038826bd9ebd'), 8, 0), (ConstBitStream('0x86fd6c5e184419'), 8, 0), (ConstBitStream('0x513c838dc1a939'), 5, 0), (ConstBitStream('0x820c74383d3728'), 5, 0), (ConstBitStream('0xa0ad038deba689'), 5, 0), (ConstBitStream('0xbc550a8a801bbc'), 5, 0), (ConstBitStream('0x87e310a0ce7c63'), 3, 0), (ConstBitStream('0xcecc04c714ed8a'), 3, 0), (ConstBitStream('0xa1f9521fb346b4'), 7, 0), (ConstBitStream('0xe1ddc50f3a8ec3'), 6, 0), (ConstBitStream('0x470b1999c84ba4'), 5, 0), (ConstBitStream('0xb6c64b5d576521'), 7, 0), (ConstBitStream('0x7bd86a53268e4a'), 7, 0), (ConstBitStream('0x3f5f7b29daef02'), 3, 0), (ConstBitStream('0x0cc625687389e5'), 6, 0), (ConstBitStream('0x982b61a1766e36'), 8, 0), (ConstBitStream('0x47c6c0fc0b09f2'), 2, 0), (ConstBitStream('0x0372fcf2bce353'), 6, 0), (ConstBitStream('0x43a02a7ca70211'), 7, 0), (ConstBitStream('0xeb3

### `3-  On Selecting Parents:`
Write your code in `select_parents` in class `GeneticAlgorithm`. Here, you'll make pairs of individuals in your randomly created population. So, if your population size is 64, you'll make 32 pairs. Any consecutive two elements in population will form one pair. These pairs are parents which will `recombine` in the next stage to reproduce offspring.

`Note`: One pair (2 individuals) will recombine to produce 2 offspring. Parents will be replaced by offspring. So, your population size will not change through out the evolution. 

See [Slides](https://drive.google.com/file/d/1exXbHn_k0_jNRbRbyVrh6Ecn2SHVEvtQ/view?usp=sharing) for further information. 

`Test` your implementation of `select_parents` by running the next cell.

In [6]:
def select_parents_sols_test(select_parents):
    genetic_algorithm = GA.GeneticAlgorithm(64, CHROMOSOME_LENGTH, verbose)
    solutions = genetic_algorithm.generate_candidate_sols(64)
    parents = evaluate_candidate_sols(genetic_algorithm, solutions)
    test.equal(len(select_parents(genetic_algorithm, parents)), (32))
    test.equal(isinstance(select_parents(genetic_algorithm, parents)[0], \
                          tuple), True)
    genetic_algorithm = GA.GeneticAlgorithm(512, CHROMOSOME_LENGTH, verbose)
    solutions = genetic_algorithm.generate_candidate_sols(512)
    parents = evaluate_candidate_sols(genetic_algorithm, solutions)
    test.equal(len(select_parents(genetic_algorithm, parents)), (256))
    test.equal(isinstance(select_parents(genetic_algorithm, parents)[0], \
                          tuple), True)

@test
def select_parents_sols(genetic_algorithm, parents):
    return genetic_algorithm.select_parents(parents)

Fitness: [(ConstBitStream('0xdc0434f0bf05be'), 5, 0), (ConstBitStream('0xd3d2ac513e2833'), 4, 0), (ConstBitStream('0x023a38b0cb1b1b'), 5, 0), (ConstBitStream('0x40d101f2164622'), 9, 0), (ConstBitStream('0x6818e6be9d2f63'), 7, 0), (ConstBitStream('0x422b60065be36f'), 5, 0), (ConstBitStream('0x1090ee032b3adb'), 5, 0), (ConstBitStream('0x893b92b74184f2'), 8, 0), (ConstBitStream('0x94b2ee73616999'), 10, 0), (ConstBitStream('0x573e223300658d'), 6, 0), (ConstBitStream('0x0f951eedd6d975'), 7, 0), (ConstBitStream('0x1793e362da2952'), 7, 0), (ConstBitStream('0xa8c60b4fbaf91b'), 2, 0), (ConstBitStream('0x253ba5d07ff87e'), 8, 0), (ConstBitStream('0x070a6aa76e06e1'), 8, 0), (ConstBitStream('0xfe163aeaf255f7'), 7, 0), (ConstBitStream('0x016c7de7a7610e'), 8, 0), (ConstBitStream('0xbea8105f07fcec'), 3, 0), (ConstBitStream('0x365db743899010'), 11, 0), (ConstBitStream('0x061c2a612324dd'), 7, 0), (ConstBitStream('0xf0e07bcb349efa'), 4, 0), (ConstBitStream('0xaa1ab3eba3f1cb'), 5, 0), (ConstBitStream('0x9

### `4- On Recombining Candidates:`
Here you'll fill in the function:
```Python
    def recombine_pairs_of_parents(self, p1, p2):
        """
        args: p1, ConstBitStream
              p2, ConstBitStream
        returns: 
        split at .6-.9 of 56 bits (CHROMOSOME_LENGTH). i.e. between 31-50 bits
        
        """
        pass
```
For recombination of a pair, you'll choose split point randomly in the range $60-90$ percent of the chromosome length (which is the length of WallBitString: 56), and return the recombined pair (offspring). 

- You may find [random range](https://docs.python.org/3/library/random.html#random.randrange) function useful. <br> 
- You may also find [Bit Stream](https://pythonhosted.org/bitstring/bitstream.html#bitstring.BitStream) helpful. 

See [Slides](https://drive.google.com/file/d/1exXbHn_k0_jNRbRbyVrh6Ecn2SHVEvtQ/view?usp=sharing) for more information. 

Test your implementation of `recombine_pairs_of_parents` by running the cell below.

In [7]:
# Test your recombine_pairs_of_parents() function
def recombine_parents_sols_test(recombine_parents_sols):
    genetic_algorithm = GA.GeneticAlgorithm(64, CHROMOSOME_LENGTH, verbose)
    solutions = genetic_algorithm.generate_candidate_sols(64)
    parents = genetic_algorithm.evaluate_candidates(solutions)
    test.equal(len(recombine_parents_sols(genetic_algorithm, parents[0], parents[1])), (2))
    if parents[0] != parents[10]:
        test.not_equal((recombine_parents_sols(genetic_algorithm, parents[0], parents[10])), (parents[0], parents[1]))
    else:
        test.equal((recombine_parents_sols(genetic_algorithm, parents[0], parents[10])), (parents[0], parents[1]))
    
    genetic_algorithm = GA.GeneticAlgorithm(512, CHROMOSOME_LENGTH, verbose)
    solutions = genetic_algorithm.generate_candidate_sols(512)
    parents = genetic_algorithm.evaluate_candidates(solutions)
    test.equal(len(recombine_parents_sols(genetic_algorithm, parents[0], parents[1])), (2))
    if parents[0] != parents[10]:
        test.not_equal((recombine_parents_sols(genetic_algorithm, parents[0], parents[10])), (parents[0], parents[1]))
    else:
        test.equal((recombine_parents_sols(genetic_algorithm, parents[0], parents[10])), (parents[0], parents[1]))
    test.equal(isinstance(recombine_parents_sols(genetic_algorithm, parents[0], parents[10])[0], \
                          bitstring.ConstBitStream), True)
    test.equal(isinstance(recombine_parents_sols(genetic_algorithm, parents[1], parents[10])[0], \
                          bitstring.ConstBitStream), True)
    
@test
def recombine_parents_sols(genetic_algorithm, parent1, parent2):
    return genetic_algorithm.recombine_pairs_of_parents(parent1, parent2)



Fitness: [(ConstBitStream('0x04874217be23db'), 3, 0), (ConstBitStream('0x798b4808596f52'), 7, 0), (ConstBitStream('0xe3c032a7882ea4'), 4, 0), (ConstBitStream('0xeec00e85fd1565'), 7, 0), (ConstBitStream('0x24bb6c5cada116'), 7, 0), (ConstBitStream('0xb0dbe124fe788b'), 5, 0), (ConstBitStream('0xe115658deebc12'), 7, 0), (ConstBitStream('0x79ab20e4fb228f'), 4, 0), (ConstBitStream('0x5374ad27b8ca6b'), 7, 0), (ConstBitStream('0x1e5d38ef357cfd'), 9, 0), (ConstBitStream('0x2dcb31adf87890'), 6, 0), (ConstBitStream('0x9ee43cc5b49c46'), 7, 0), (ConstBitStream('0x910e323448fe44'), 9, 0), (ConstBitStream('0xb9144a390d8b74'), 6, 0), (ConstBitStream('0x783e0aeb054105'), 5, 0), (ConstBitStream('0x61ae6dd5ce557c'), 7, 0), (ConstBitStream('0xa2543af6fedca5'), 9, 0), (ConstBitStream('0x7cdcd2c7df4298'), 5, 0), (ConstBitStream('0x8e5851b7ee6507'), 10, 0), (ConstBitStream('0x5262bdb42dfb02'), 9, 0), (ConstBitStream('0xd03438cac6f3f6'), 5, 0), (ConstBitStream('0x80daaa5fde7884'), 6, 0), (ConstBitStream('0x7b

### TESTING recombine_parents_sols: PASSED 6/6
###



### `5- On Mutating Offspring:`
Here, you'll fill in the function:
```Python
def mutate_offspring(self, p):
       '''
       args: individual (ConstBitStream)
       returns: individual (ConstBitStream)
       '''
       pass
```
This function takes as argument only one individual.
You'll flip each bit (0 => 1 OR 1 => 0) of the BitString independently with a "certain" probability called mutation rate- defined in [Slides](https://drive.google.com/file/d/1exXbHn_k0_jNRbRbyVrh6Ecn2SHVEvtQ/view?usp=sharing).

- You may find [random](https://docs.python.org/3/library/random.html#module-random) helpful. 
- You may also find [Bit Stream](https://pythonhosted.org/bitstring/bitstream.html#bitstring.BitStream) helpful. 


Test your implementation of `mutate_offspring` by running the cell below.

In [8]:
# Test your mutate_offspring() function
def mutate_offspring_sols_test(mutate_offspring_sols):
    genetic_algorithm = GA.GeneticAlgorithm(64, CHROMOSOME_LENGTH, verbose)
    solutions = genetic_algorithm.generate_candidate_sols(64)
    parents = genetic_algorithm.evaluate_candidates(solutions)
    test.equal(len(mutate_offspring_sols(genetic_algorithm, parents[0])), CHROMOSOME_LENGTH)
    test.equal(isinstance(mutate_offspring_sols(genetic_algorithm, parents[0]), \
                          bitstring.ConstBitStream), True)
    
@test
def mutate_offspring_sols(genetic_algorithm, parent):
    return genetic_algorithm.mutate_offspring(parent)


Fitness: [(ConstBitStream('0x1401aa05b0ae01'), 7, 0), (ConstBitStream('0xadfb6e8a600481'), 7, 0), (ConstBitStream('0xc55f27af10b21d'), 8, 0), (ConstBitStream('0x219317709cfb10'), 6, 0), (ConstBitStream('0x24310c0263d892'), 7, 0), (ConstBitStream('0x5f9177aa7a7882'), 8, 0), (ConstBitStream('0xf6da9ee130b53c'), 5, 0), (ConstBitStream('0xd7fd71baaa0f66'), 10, 0), (ConstBitStream('0xfd9efa961ab717'), 8, 0), (ConstBitStream('0x502a03b63e79c3'), 6, 0), (ConstBitStream('0xc68f353a3a3c7d'), 8, 0), (ConstBitStream('0x6f480db4beafb8'), 5, 0), (ConstBitStream('0x54c65a45d7c863'), 6, 0), (ConstBitStream('0x161a9a67ac5ca5'), 12, 0), (ConstBitStream('0x44c32b8bee7492'), 5, 0), (ConstBitStream('0x5b3e4752406476'), 9, 0), (ConstBitStream('0xc19e501847bbd8'), 5, 0), (ConstBitStream('0x064872d04c258d'), 7, 0), (ConstBitStream('0x946ff76868cb9b'), 9, 0), (ConstBitStream('0x1d5b7f77f4c60b'), 12, 0), (ConstBitStream('0x2c29f5a7b15aa3'), 10, 0), (ConstBitStream('0x2902dbd93d18f6'), 4, 0), (ConstBitStream('0

## `On Converging to the Fittest Individual:`
This is the real test. Your solution should find a candidate with 100% solution in a few minutes (not greater than 10 minutes).

In [ ]:
resp = input('Do you want to see Fitness of each candidate solution? yes/no ')
if resp in ['yes', 'y', '1', 'Y']:
    verbose = True

genetic_algorithm = GA.GeneticAlgorithm(POPULATION_SIZE, CHROMOSOME_LENGTH, verbose)
genetic_algorithm.run_genetic_alg()

Do you want to see Fitness of each candidate solution? yes/no y
Fitness: [(ConstBitStream('0x1d13a56bcbded5'), 12, 0), (ConstBitStream('0xb34b242cca5135'), 7, 0), (ConstBitStream('0x1e33fc32e802dc'), 7, 0), (ConstBitStream('0x5fe52cd27214cf'), 6, 0), (ConstBitStream('0xe8d9be6617aa1a'), 11, 0), (ConstBitStream('0x955559aee9d165'), 14, 0), (ConstBitStream('0x1a096f7113966d'), 10, 0), (ConstBitStream('0xbada0aa082b5e5'), 5, 0), (ConstBitStream('0xe643aa46ca30cf'), 6, 0), (ConstBitStream('0x404e73125dd73d'), 8, 0), (ConstBitStream('0x0926ef2faf32ff'), 3, 0), (ConstBitStream('0xb3581aa8f7295c'), 6, 0), (ConstBitStream('0x7d468730072758'), 9, 0), (ConstBitStream('0x384c26d0bf8dac'), 2, 0), (ConstBitStream('0xf2918beda82af7'), 6, 0), (ConstBitStream('0x56cf10f8561c29'), 7, 0), (ConstBitStream('0xb791b22cce7c3d'), 7, 0), (ConstBitStream('0xcfc3788c0a7531'), 4, 0), (ConstBitStream('0xf319885ef4ee07'), 9, 0), (ConstBitStream('0x0d2593fa08ba3b'), 6, 0), (ConstBitStream('0x9905235bd1a49d'), 8, 0)

Fitness: [(ConstBitStream('0x7482d843f0c574'), 5, 0), (ConstBitStream('0x3076c61f30f8f7'), 3, 0), (ConstBitStream('0xa819c793701237'), 8, 0), (ConstBitStream('0x40d14f07f1f2c2'), 5, 0), (ConstBitStream('0x14bbc7603e6812'), 7, 0), (ConstBitStream('0x2fe7e8e93db1a2'), 1, 0), (ConstBitStream('0x786ff7348a4d7d'), 8, 0), (ConstBitStream('0xf77e0b2df4a9da'), 4, 0), (ConstBitStream('0x130883e1c98a7f'), 6, 0), (ConstBitStream('0x42b6f8b826bff5'), 6, 0), (ConstBitStream('0x80cc612773fe57'), 7, 0), (ConstBitStream('0xe979c4ff2af60a'), 9, 0), (ConstBitStream('0x2bbaa6a23c0cf7'), 5, 0), (ConstBitStream('0xfd44a7f2ade715'), 12, 0), (ConstBitStream('0x1767e913cbc751'), 8, 0), (ConstBitStream('0x4ba251ce568fd7'), 6, 0), (ConstBitStream('0x191d02d3da2efb'), 5, 0), (ConstBitStream('0x9e4d1b4d2b998a'), 6, 0), (ConstBitStream('0x285968d74d93f2'), 8, 0), (ConstBitStream('0x3ca6f7572ea3f1'), 6, 0), (ConstBitStream('0xf1a2b7dea37a48'), 9, 0), (ConstBitStream('0xf77ff0da7e53c7'), 8, 0), (ConstBitStream('0x3b

Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  7.14 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Fitness: [(ConstBitStream('0x4ff4a901174420'), 5, 0), (ConstBitStream('0xb46382a21e0105'), 6, 0), (ConstBitStream('0x65d630631e67ac'), 7, 0), (ConstBitStream('0x9b161d8fe251c1'), 7, 0), (ConstBitStream('0x1b2ba2f34b3547'), 8, 0), (ConstBitStream('0x4b3357f10c0667'), 6, 0), (ConstBitStream('0x9ec83145d86925'), 9, 0), (ConstBitStream('0xd14cddca6ae3a6'), 8, 0), (ConstBitStream('0x8de7fccd6bdc43'), 6, 0), (ConstBitStream('0xdafcb6dc59c4ed'), 8, 0), (ConstBitStream('0x57f248b5fc474b'), 7, 0), (ConstBitStream('0xcfd87a2ee4d9b5'), 9, 0),

Fitness: [(ConstBitStream('0x7391022c99ba62'), 5, 0), (ConstBitStream('0x495b02c77edc4d'), 9, 0), (ConstBitStream('0xd6e471c709fa3a'), 7, 0), (ConstBitStream('0x26b0cda6783c0b'), 5, 0), (ConstBitStream('0x41f975778f23d4'), 11, 0), (ConstBitStream('0x76f8f4668d8c7d'), 10, 0), (ConstBitStream('0x8b0deaed15b17c'), 4, 0), (ConstBitStream('0x19df231508e528'), 5, 0), (ConstBitStream('0x190dd4efe20393'), 6, 0), (ConstBitStream('0xba5f8568363ea5'), 10, 0), (ConstBitStream('0xf961d446d70c51'), 11, 0), (ConstBitStream('0x27d1913412a3e1'), 8, 0), (ConstBitStream('0x38b891e54c487d'), 8, 0), (ConstBitStream('0x007308b9bae80b'), 2, 0), (ConstBitStream('0x3ee8717cf16592'), 9, 0), (ConstBitStream('0xaac7758db67bca'), 5, 0), (ConstBitStream('0x86352ba39e1fa0'), 3, 0), (ConstBitStream('0x0da04ac1f71d05'), 5, 0), (ConstBitStream('0xcf3aaff4859dcf'), 7, 0), (ConstBitStream('0x68f348d9d25521'), 4, 0), (ConstBitStream('0x53b0d699e8ac7f'), 4, 0), (ConstBitStream('0xc6f051f712fa6a'), 6, 0), (ConstBitStream('0

Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness: 

...

Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  14.29 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Fitness: [(ConstBitStream('0xd73d7eff79d5f1'), 10, 0), (ConstBitStream('0xa69802fe9ae0c6'), 6, 0), (ConstBitStream('0x0b9261aed78679'), 8, 0), (ConstBitStr

Fitness: [(ConstBitStream('0x4009bff029f16e'), 8, 0), (ConstBitStream('0x710ffbc9e93686'), 7, 0), (ConstBitStream('0x33126231bb4db5'), 5, 0), (ConstBitStream('0x8ea4ad8a8a633d'), 5, 0), (ConstBitStream('0x00409d727657ad'), 12, 0), (ConstBitStream('0x9422f6c8fc3fcb'), 4, 0), (ConstBitStream('0x8d3e8d56e2b9e9'), 8, 0), (ConstBitStream('0xd675ce56a68700'), 10, 0), (ConstBitStream('0x44475051b8dfea'), 4, 0), (ConstBitStream('0xa283bcd4379908'), 5, 0), (ConstBitStream('0xb96c3f8f285ec5'), 9, 0), (ConstBitStream('0x07dad41c399661'), 10, 0), (ConstBitStream('0x977729ee24d954'), 11, 0), (ConstBitStream('0x7bdb0a74b3b8d3'), 6, 0), (ConstBitStream('0x36a269cf757cfc'), 6, 0), (ConstBitStream('0x7485b28379d8b4'), 8, 0), (ConstBitStream('0x57a0e892c0dbf0'), 4, 0), (ConstBitStream('0x8b30797c1d0cae'), 5, 0), (ConstBitStream('0x12fcc73476b634'), 9, 0), (ConstBitStream('0x6d61b942b5070e'), 11, 0), (ConstBitStream('0xe5b3658e7b5ef6'), 10, 0), (ConstBitStream('0xa14dc58b92b80f'), 5, 0), (ConstBitStream(

Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  

Fitness: [(ConstBitStream('0x0dcafb3b893763'), 7, 0), (ConstBitStream('0xb5c45628f87e9b'), 6, 0), (ConstBitStream('0x3c1b9c4ca97175'), 10, 0), (ConstBitStream('0xaacbe281068f35'), 4, 0), (ConstBitStream('0x0f734d6add7da4'), 7, 0), (ConstBitStream('0xb8ff4af839cc77'), 4, 0), (ConstBitStream('0xe37fdcab3acfaf'), 1, 0), (ConstBitStream('0x16b303cb8639f7'), 4, 0), (ConstBitStream('0x800d820421e1ed'), 6, 0), (ConstBitStream('0xc9da25e9deef26'), 6, 0), (ConstBitStream('0x1bd9f6bbb8a10a'), 7, 0), (ConstBitStream('0xc74918a90093a9'), 6, 0), (ConstBitStream('0x32fa6f3be00058'), 5, 0), (ConstBitStream('0x6fc85a787c0849'), 6, 0), (ConstBitStream('0x80573eeb85b85b'), 7, 0), (ConstBitStream('0x2f195b31a52f52'), 9, 0), (ConstBitStream('0x3212c6c71ac3e4'), 4, 0), (ConstBitStream('0xedf567af9de397'), 6, 0), (ConstBitStream('0xdc1381a59535e5'), 10, 0), (ConstBitStream('0x8e09839a221649'), 10, 0), (ConstBitStream('0x1a932d9f90339f'), 4, 0), (ConstBitStream('0x99466d554c5f2a'), 9, 0), (ConstBitStream('0x

Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  3.57 %
Individual Fitness:  3.57 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Individual Fitness:  0.0 %
Fitness: [(ConstBitStream('0xcb69e1459b09fd'), 7, 0), (ConstBitStream('0xfb833ff35e6f92'), 5, 0), (ConstBitStream('0x32df641ba2bb8f'), 3, 0), (ConstBitStream('0x488d1369d111c9'), 5, 0), (ConstBitStream('0x54e3b204c4add3'), 7, 0), (ConstBitStream('0x60fd0689fadc74'), 5, 0), (ConstBitStream('0x44fe0af5a6fb10'), 6, 0), (ConstBitStream('0x1e857d261d9d77'), 10, 0), (ConstBitStream('0xe7252391828aa8'), 3, 0), (ConstBitStream('0x2eccb1349d00bd'), 7, 0), (ConstBitStream('0x0dc8191d221a54'), 9, 0), (ConstBitStream('0x5a72a574df15b3'), 10, 0), (ConstBitStream('0x45607452a66c21'), 11, 0), (ConstBitStream('0x8214c8a555ee66'), 8, 0), (ConstBitStream('0x5cf314e854a8ea'), 4, 0), (ConstBitStream('0x69544bc6089ac7'), 8, 0), (ConstBitSt